In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import sklearn.metrics as sm
from scipy.sparse import hstack

In [2]:
# class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

total = pd.read_csv('./data/newAPIs2.csv').fillna(' ')
# test = pd.read_csv('../data/test.csv').fillna(' ')

domain = pd.read_csv('./data/tagnet.csv')

class_names = domain['source'].drop_duplicates().values

test = total.sample(frac=0.2,axis=0,random_state=0)
train = total[~total['id'].isin(test['id'].values)]


train_text = train['descr']
test_text = test['descr']
all_text = pd.concat([train_text, test_text])

In [ ]:
labels = dict()
for cla in tqdm(class_names):
    clist = list()
    for idx, row in train.iterrows():
        if row['tags2'].find(cla) == -1:
            clist.append(0)
        else:
            clist.append(1)
    labels[cla] = clist


 73%|██████████████████████████████████████████████████████████▏                     | 343/472 [03:24<01:16,  1.68it/s]

In [ ]:
label2 = labels.copy()
class_names2 = list(class_names) 

In [ ]:
key = list(labels.keys())
for k in key:
    if sum(labels[k]) < 100:
        labels.pop(k)
#         print(k)
#         class_names2.remove(k)

In [ ]:
class_names2 = list(labels.keys())
len(class_names2)

In [3]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)



char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

c:\users\zyz\appdata\local\programs\python\python36\lib\site-packages\sklearn\feature_extraction\text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


In [4]:
train_features

<10335x60000 sparse matrix of type '<class 'numpy.float64'>'
	with 14441400 stored elements in COOrdinate format>

In [61]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in tqdm(class_names2):
    train_target = labels[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

  0%|                                                                                           | 0/97 [00:00<?, ?it/s]

CV score for class Mapping is 0.9631280919365904


  1%|▊                                                                                  | 1/97 [00:09<15:11,  9.50s/it]

CV score for class Other is 0.7532929400414771


  2%|█▋                                                                                 | 2/97 [00:19<15:20,  9.68s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 95/97 [16:33<00:20, 10.45s/it]

CV score for class Language is 0.9601633049332001


 99%|█████████████████████████████████████████████████████████████████████████████████▏| 96/97 [16:42<00:10, 10.44s/it]

CV score for class Translation is 0.9841853375458739


100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [16:51<00:00, 10.43s/it]


In [62]:
submission

,id,Mapping,Other,Data,Tools,Design,eCommerce,Authentication,Education,Games,...,Food,Reference,England,Time,Jobs,Chat,Blogging,Auto,Language,Translation
12648,12648,0.059502,0.016300,0.069075,0.184158,0.010389,0.040255,0.011137,0.020989,0.019257,...,0.009184,0.024092,0.013664,0.068783,0.010350,0.012917,0.011517,0.014190,0.020858,0.010894
7431,7431,0.043236,0.012561,0.099321,0.092043,0.009730,0.054804,0.011015,0.023659,0.023198,...,0.010060,0.044418,0.014443,0.025039,0.009756,0.009785,0.011265,0.017856,0.021026,0.008876
7707,7707,0.045695,0.015220,0.072321,0.212006,0.012010,0.041631,0.011339,0.024081,0.019779,...,0.009700,0.034563,0.014028,0.021239,0.009491,0.020062,0.013947,0.015876,0.027506,0.009500
3028,3028,0.039052,0.012363,0.181918,0.041233,0.008606,0.056111,0.009688,0.021292,0.068832,...,0.009558,0.041040,0.016065,0.039505,0.009902,0.010315,0.010090,0.021476,0.016918,0.008120
3130,3130,0.044174,0.014767,0.065899,0.079825,0.010228,0.078384,0.012327,0.021850,0.019878,...,0.010541,0.034765,0.014030,0.023662,0.010546,0.010133,0.010483,0.018398,0.020043,0.009998
8544,8544,0.057848,0.015260,0.207436,0.081906,0.010276,0.107057,0.011117,0.027027,0.021142,...,0.012621,0.105966,0.018663,0.020650,0.009479,0.009626,0.011008,0.018625,0.017722,0.008505
10118,10118,0.042728,0.014360,0.082552,0.134707,0.010507,0.046576,0.012218,0.028602,0.021358,...,0.010588,0.056406,0.015187,0.019918,0.010334,0.009952,0.012094,0.015939,0.023266,0.009544
9102,9102,0.040676,0.013522,0.109495,0.061912,0.010939,0.110587,0.010455,0.023828,0.019467,...,0.010069,0.048789,0.017624,0.023877,0.009535,0.010882,0.011468,0.020430,0.019241,0.008671
5040,5040,0.045315,0.016867,0.089092,0.152474,0.011858,0.042891,0.012122,0.024121,0.019690,...,0.009232,0.031391,0.012978,0.020915,0.009430,0.012078,0.013444,0.017971,0.032514,0.010745
296,296,0.037780,0.013869,0.108852,0.093276,0.010625,0.052787,0.019808,0.023031,0.020817,...,0.008940,0.026124,0.012333,0.021877,0.010346,0.011770,0.011097,0.017063,0.019753,0.008916


In [63]:
gt_labels = dict()
for cla in tqdm(class_names2):
    clist = list()
    for idx, row in test.iterrows():
        if row['tags2'].find(cla) == -1:
            clist.append(0)
        else:
            clist.append(1)
    gt_labels[cla] = clist

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [00:15<00:00,  6.24it/s]


In [64]:
mAP = list()
for cla in tqdm(class_names2):
    ap = sm.average_precision_score(gt_labels[cla],submission[cla])
    mAP.append(ap)
np.mean(mAP)

100%|█████████████████████████████████████████████████████████████████████████████████| 97/97 [00:00<00:00, 372.64it/s]


0.5550490386566358

In [ ]:

# MAP
# 0.501658837537489  -- 大于50 的 178个tag 
# 0.5550490386566358 -- 大于100 的 97个tag